In [1]:
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline
pd.set_option('display.max_columns', None)
sns.set()

In [2]:
demo_df = pd.read_excel("../data/demographics 8-15-16.xlsx")
print demo_df.shape
demo_df.head()

(944, 28)


,PIN,Status,Gender,D.O.B.,Age,Language,Living Set-Up,Medicaid,Hospitalized,RPM Hospitalized,RPM Falls,Rehospitalized,RPM Rehospitalized,CHF,COPD,Diabetes,Hypertension,Stroke,Hypoglycemia,Falls,Liver Disease,Hypotension,Obesity,Coronary Artery Disease,Fractures,Renal,Alzheimer's,Depression
0,10001,A,F,12-22-1940,75.698630,English,Family No CG,N,Y,Y,Y,N,N,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
1,10002,A,F,04-24-1933,83.367123,English,Alone No CG,N,N,NaN,Y,N,NaN,NaN,X,X,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
2,10003,D,F,08-18-1920,96.057534,English,Family No CG,Y,N,NaN,NaN,N,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10004,A,F,12-15-1948,67.712329,English,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10005,D,F,02-18-1934,82.545205,English,NaN,N,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Drop duplicate PINS
print demo_df.shape
demo_df.drop_duplicates(['PIN'],keep=False,inplace=True)
print demo_df.shape

demo_df.rename(columns=lambda x: x.lower().replace(" ","_"),inplace=True)
demo_df.rename(columns={"alzheimer's":'alzheimers'},inplace=True)
print demo_df.columns

